# Ноутбук 02: Data Preprocessing

**Цель:**

- Подготовить данные для обучения моделей;

- Закодировать последовательности в числа;

- Применить padding;

- Разделить на train/val/test;

- Сохранить результаты.

In [ ]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from collections import Counter

## Загрузка данных

In [42]:
df = pd.read_csv("../data/raw/2022-08-03-ss.cleaned.csv")

## Создание словарей

In [ ]:
AMINO_ACIDS = 'ACDEFGHIKLMNPQRSTVWY'
aa_to_int = {aa: i+1 for i, aa in enumerate(AMINO_ACIDS)}
aa_to_int['<PAD>'] = 0
int_to_aa = {v: k for k, v in aa_to_int.items()}


STRUCTURES = 'HEC'
ss_to_int = {ss: i+1 for i, ss in enumerate(STRUCTURES)}
ss_to_int['<PAD>'] = 0
int_to_ss = {v: k for k, v in ss_to_int.items()}


print(f"\nСловарь aa_to_int: {aa_to_int}")
print(f"\nСловарь ss_to_int: {ss_to_int}")


Словарь aa_to_int: {'A': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'K': 9, 'L': 10, 'M': 11, 'N': 12, 'P': 13, 'Q': 14, 'R': 15, 'S': 16, 'T': 17, 'V': 18, 'W': 19, 'Y': 20, '<PAD>': 0}

Словарь ss_to_int: {'H': 1, 'E': 2, 'C': 3, '<PAD>': 0}


## Функции кодирования

In [ ]:
def encode_sequence(seq, vocab):
    """Кодирует строку в список чисел"""
    return [vocab.get(char, 0) for char in seq]

def decode_sequence(encoded, vocab_reverse):
    """Декодирует числа обратно в строку"""
    return ''.join([vocab_reverse.get(i, '<PAD>') for i in encoded])

## Padding функция

In [45]:
def pad_sequence(seq, max_length, pad_value=0):
    """Применяет padding или truncation"""
    if len(seq) > max_length:
        # Обрезаем
        return np.array(seq[:max_length])
    else:
        # Дополняем нулями
        padded = np.zeros(max_length, dtype=int)
        padded[:len(seq)] = seq
        return padded

##  Выбор max_length

In [46]:
max_length = 700


coverage = (df['len'] <= max_length).sum() / len(df) * 100
print(f"max_length={max_length} покрывает {coverage:.2f}% данных")
print(f"Будет обрезано {len(df[df['len'] > max_length])} последовательностей")

max_length=700 покрывает 96.48% данных
Будет обрезано 16798 последовательностей


## Обработка всего датасета

In [47]:
X_list = []
y_list = []

for idx, row in df.iterrows():
    # Кодируем последовательность
    seq_encoded = encode_sequence(row['seq'], aa_to_int)
    seq_padded = pad_sequence(seq_encoded, max_length)
    
    # Кодируем структуру
    struct_encoded = encode_sequence(row['sst3'], ss_to_int)
    struct_padded = pad_sequence(struct_encoded, max_length)
    
    X_list.append(seq_padded)
    y_list.append(struct_padded)


X = np.array(X_list)
y = np.array(y_list)

print(f"Форма X (последовательности): {X.shape}")
print(f"Форма y (структуры): {y.shape}")

Форма X (последовательности): (477153, 700)
Форма y (структуры): (477153, 700)


## Разделение на train/val/test с учётом дисбаланса классов

In [48]:
df['dominant_class'] = df['sst3'].apply(
    lambda s: max(['H', 'E', 'C'], key=lambda c: s.count(c))
)

# Stratified split
dominant_classes = df['dominant_class'].values

X_temp, X_test, y_temp, y_test, _, _ = train_test_split(
    X, y, dominant_classes,
    test_size=0.1, 
    random_state=42,
    stratify=dominant_classes
)

X_train, X_val, y_train, y_val, _, _ = train_test_split(
    X_temp, y_temp, dominant_classes[:len(X_temp)],
    test_size=0.111,
    random_state=42,
    stratify=dominant_classes[:len(X_temp)]
)

# Вычисляем class weights
all_labels = y_train[y_train != 0]
class_weights = compute_class_weight(
    'balanced',
    classes=np.array([1, 2, 3]),
    y=all_labels
)

# Сохраняем всё
np.save('../data/processed/class_weights.npy', class_weights)

In [49]:
def check_distribution(classes, name):
    counts = Counter(classes)
    total = len(classes)
    print(f"\n{name}:")
    for cls in ['H', 'E', 'C']:
        print(f"  {cls}: {counts[cls]} ({counts[cls]/total*100:.1f}%)")

check_distribution(dominant_classes, "Весь датасет")
check_distribution(classes_train, "Train")
check_distribution(classes_val, "Val")
check_distribution(classes_test, "Test")


Весь датасет:
  H: 152988 (32.1%)
  E: 44311 (9.3%)
  C: 279854 (58.7%)

Train:
  H: 122406 (32.1%)
  E: 35453 (9.3%)
  C: 223910 (58.7%)

Val:
  H: 15283 (32.1%)
  E: 4427 (9.3%)
  C: 27958 (58.7%)

Test:
  H: 15299 (32.1%)
  E: 4431 (9.3%)
  C: 27986 (58.7%)


In [50]:
print(f"\nClass weights:")
print(f"  H (1): {class_weights[0]:.3f}")
print(f"  E (2): {class_weights[1]:.3f}")
print(f"  C (3): {class_weights[2]:.3f}")


Class weights:
  H (1): 0.980
  E (2): 1.614
  C (3): 0.735


## Создание масок

In [51]:
def create_mask(sequences):
    """Создаёт маску: 1 для валидных позиций, 0 для padding"""
    return (sequences != 0).astype(int)

# Создаём маски для всех наборов
mask_train = create_mask(X_train)
mask_val = create_mask(X_val)
mask_test = create_mask(X_test)

print(f"Форма mask_train: {mask_train.shape}")

Форма mask_train: (381769, 700)


## Сохранение данных

In [ ]:
import os

# Создаём папку если не существует
os.makedirs('../data/processed', exist_ok=True)

# Сохраняем массивы
np.save('../data/processed/X_train.npy', X_train)
np.save('../data/processed/y_train.npy', y_train)
np.save('../data/processed/X_val.npy', X_val)
np.save('../data/processed/y_val.npy', y_val)
np.save('../data/processed/X_test.npy', X_test)
np.save('../data/processed/y_test.npy', y_test)

# Сохраняем маски
np.save('../data/processed/mask_train.npy', mask_train)
np.save('../data/processed/mask_val.npy', mask_val)
np.save('../data/processed/mask_test.npy', mask_test)

# Сохраняем словари
vocabularies = {
    'aa_to_int': aa_to_int,
    'int_to_aa': int_to_aa,
    'ss_to_int': ss_to_int,
    'int_to_ss': int_to_ss,
    'max_length': max_length
}

with open('../data/processed/vocabularies.pkl', 'wb') as f:
    pickle.dump(vocabularies, f)

print("Все данные сохранены в data/processed/")

✅ Все данные сохранены в data/processed/


## Проверка результатов

In [55]:
# Проверяем что можем загрузить
X_train_loaded = np.load('../data/processed/X_train.npy')
print(f"X_train загружен: {X_train_loaded.shape}")

for i in range(1):
    print(f"\nПример {i+1}")
    
    # Декодируем обратно
    seq_decoded = decode_sequence(X_train[i], int_to_aa)
    struct_decoded = decode_sequence(y_train[i], int_to_ss)
    
    # Убираем padding
    length = mask_train[i].sum()
    seq_clean = seq_decoded[:length]
    struct_clean = struct_decoded[:length]
    
    print(f"Длина: {length}")
    print(f"Sequence: {seq_clean[:60]}...")
    print(f"Structure: {struct_clean[:60]}...")
    print(f"Encoded X (первые 10): {X_train[i][:10]}")
    print(f"Encoded y (первые 10): {y_train[i][:10]}")


X_train загружен: (381769, 700)

Пример 1
Длина: 257
Sequence: MLSAFQLENNRLTRLEVEESQPLVNAVWIDLVEPDDDERLRVQSELGQSLATRPELEDIE...
Structure: CEEEEEECCCCEEECCCCCCCCCCCCCEEEEECCCHHHHHHHHHHCCCCCCCHHHHCCCC...
Encoded X (первые 10): [11 10 16  1  5 14 10  4 12 12]
Encoded y (первые 10): [3 2 2 2 2 2 2 3 3 3]


# Выводы:

### 1. Размеры обработанных данных

**Успешно подготовлено:**
- **Train:** 381,769 белков (80%)
- **Validation:** 47,668 белков (10%)
- **Test:** 47,716 белков (10%)
- **Всего:** 477,153 белковых последовательности

**Форма данных:**
- `X` (последовательности): (N, 700) — закодированные аминокислоты
- `y` (структуры): (N, 700) — закодированные метки H/E/C
- `mask` (маски): (N, 700) — индикаторы валидных позиций

***

### 2. Параметры кодирования

**Словари:**
- **Аминокислоты:** 21 токен (20 стандартных + padding)
  - A→1, C→2, D→3, ..., Y→20, `<PAD>`→0
- **Структуры:** 4 класса
  - H (Helix)→1, E (Sheet)→2, C (Coil)→3, `<PAD>`→0

**Параметры padding:**
- `max_length = 700` — покрывает **96.48%** данных
- Обрезано: 16,798 последовательностей (3.52%) длиннее 700 аминокислот
- **Обоснование:** баланс между сохранением данных и эффективностью обучения

***

### 3. Распределение классов (Stratified Split)

**Применена стратификация по доминирующему классу каждого белка:**

| Набор | H (Helix) | E (Sheet) | C (Coil) |
|-------|-----------|-----------|----------|
| **Весь датасет** | 32.1% | 9.3% | 58.7% |
| **Train** | 32.1% | 9.3% | 58.7% |
| **Val** | 32.1% | 9.3% | 58.7% |
| **Test** | 32.1% | 9.3% | 58.7% |

**Пропорции классов сохранены во всех наборах!**

***

### 4. Class Weights для борьбы с дисбалансом

**Вычислены веса для weighted loss:**
- **H (1):** 0.980 — почти нейтральный вес
- **E (2):** 1.614 — **повышенный** вес (редкий класс, 9.3%)
- **C (3):** 0.735 — пониженный вес (частый класс, 58.7%)

**Назначение:** 
При обучении модель будет **больше штрафоваться** за ошибки на редком классе E (Sheet) и меньше на частом классе C (Coil). Это компенсирует дисбаланс классов.

**Сохранено:** `../data/processed/class_weights.npy`

***

### 5. Маски для игнорирования padding

**Созданы маски для всех наборов:**
- `mask[i, j] = 1` — валидная позиция (реальная аминокислота)
- `mask[i, j] = 0` — padding (дополнение нулями)

**Пример использования при обучении:**
```python
# Loss будет вычисляться только на валидных позициях
loss = criterion(predictions, targets)  # ignore_index=0 автоматически игнорирует padding
```

***

### 6. Проверка корректности

**Тестирование encoding/decoding:**
- ✅ Последовательности корректно кодируются в числа
- ✅ Декодирование восстанавливает исходные строки (без padding)
- ✅ Маски правильно определяют границы реальных данных

**Пример:**
```
Оригинал:  "MLSAFQLENNRL..."
Encoded:   [11, 10, 16, 1, 5, 14, ...]
Decoded:   "MLSAFQLENNRL..."  ✅ Совпадает
```

***

### 7. Сохранённые файлы

**Все данные сохранены в `../data/processed/`:**

```
data/processed/
├── X_train.npy          # Тренировочные последовательности
├── y_train.npy          # Тренировочные структуры
├── X_val.npy            # Валидационные последовательности
├── y_val.npy            # Валидационные структуры
├── X_test.npy           # Тестовые последовательности
├── y_test.npy           # Тестовые структуры
├── mask_train.npy       # Маски для train
├── mask_val.npy         # Маски для val
├── mask_test.npy        # Маски для test
├── class_weights.npy    # Веса классов для weighted loss
└── vocabularies.pkl     # Словари и параметры (max_length, aa_to_int, etc.)
```

***

### 8. Ключевые решения и обоснования

| Решение | Обоснование |
|---------|-------------|
| **max_length=700** | Покрывает 96.5% данных, оптимальный баланс |
| **Stratified split** | Сохраняет пропорции классов H/E/C во всех наборах |
| **Class weights** | Компенсирует дисбаланс: E (9.3%) получает вес 1.614 |
| **Padding token=0** | Стандартная практика, позволяет игнорировать при loss |
| **80/10/10 split** | Достаточно данных для обучения и надёжной оценки |
